In [56]:
import mysql.connector
import random
from faker import Faker
from datetime import datetime, timedelta
import csv

In [57]:
# Initialize Faker for generating random data
fake = Faker()

In [58]:
# MySQL Database Connection Details
DB_CONFIG = {
    "host": "localhost",
    "user": "root",
    "password": "12345",
    "database": "crop_management"
}

In [59]:
# Connect to MySQL
conn = mysql.connector.connect(**DB_CONFIG)
cursor = conn.cursor()

# Create crops table if not exists
cursor.execute("""
CREATE TABLE IF NOT EXISTS crops (
    id INT AUTO_INCREMENT PRIMARY KEY,
    crop_name VARCHAR(255) NOT NULL,
    planting_date DATE,
    harvest_date DATE,
    growth_stage VARCHAR(255),
    pest_control_measures TEXT,
    yield_prediction INT
)
""")
conn.commit()

In [60]:
# List of sample crop names
crop_names = ["Wheat", "Rice", "Corn", "Soybean", "Barley", "Sugarcane", "Cotton", "Potato", "Tomato", "Lettuce"]

# List of possible growth stages
growth_stages = ["Seedling", "Vegetative", "Flowering", "Fruiting", "Maturity"]

# List of sample pest control measures
pest_control_measures_list = [
    "Use of organic pesticides",
    "Crop rotation",
    "Neem oil application",
    "Biological pest control",
    "Chemical pesticides",
    "Regular field monitoring",
]

In [61]:
# Function to generate random data for crops
def generate_data():
    crop_name = random.choice(crop_names)
    planting_date = fake.date_between(start_date="-2y", end_date="today")  # Planting in last 2 years
    harvest_date = planting_date + timedelta(days=random.randint(60, 180))  # Harvest after 2-6 months
    growth_stage = random.choice(growth_stages)
    pest_control = random.choice(pest_control_measures_list)
    yield_prediction = random.randint(500, 5000)  # Yield in kg
    
    return (crop_name, planting_date, harvest_date, growth_stage, pest_control, yield_prediction)

In [62]:
# Batch insert records in chunks
batch_size = 100000  # Insert 100,000 at a time for efficiency
total_records = 200000

for i in range(0, total_records, batch_size):
    data_batch = [generate_data() for _ in range(batch_size)]
    
    # Execute batch insert
    cursor.executemany("""
        INSERT INTO crops (crop_name, planting_date, harvest_date, growth_stage, pest_control_measures, yield_prediction)
        VALUES (%s, %s, %s, %s, %s, %s)
    """, data_batch)
    conn.commit()
    
    print(f"{i + batch_size} records inserted...")

print("✅ Data insertion completed successfully!")

# Close the database connection
cursor.close()
conn.close()

100000 records inserted...
200000 records inserted...
✅ Data insertion completed successfully!


In [ ]:
import mysql.connector
import csv

# ✅ Step 1: Connect to MySQL
DB_CONFIG = {
    "user": "root",                # Your MySQL username
    "password": "12345",   # Your MySQL password
    "host": "localhost",           # Database host
    "database": "crop_management"    # Your database name
}

try:
    conn = mysql.connector.connect(**DB_CONFIG)
    cursor = conn.cursor()

    # ✅ Step 2: Execute SQL Query to Get Data
    query = "SELECT * FROM crops"
    cursor.execute(query)

    # ✅ Step 3: Fetch All Data
    rows = cursor.fetchall()
    column_names = [desc[0] for desc in cursor.description]  # Get column names

    # ✅ Step 4: Write Data to CSV
    csv_file_path = "out.csv"  # Change this to your desired path
    with open(csv_file_path, mode="w", newline="", encoding="utf-8") as file:
        writer = csv.writer(file)
        writer.writerow(column_names)  # Write column headers
        writer.writerows(rows)  # Write data rows

    print(f"✅ Data exported successfully to {csv_file_path}")

except mysql.connector.Error as e:
    print(f"❌ Error: {e}")

finally:
    if conn.is_connected():
        cursor.close()
        conn.close()
